In [ ]:
!apt-get -qq install -y libsm6 libxext6 && pip install -q -U opencv-python
import cv2

     |████████████████████████████████| 50.4MB 86kB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.


In [ ]:
import time
time.asctime()
cv2.__version__

'4.5.1'

In [ ]:
import sys
import time
import kafka import kafkaProducer
import numpy as np
import cv2

# Define the codec and create VideoWriter object (note isColor is False for Gray)
fourcc = cv2.VideoWriter_fourcc(*'DIVX')
out = cv2.VideoWriter('motiondetect.avi',fourcc, 60.0, (1920,1080), isColor = False)
#MOG2 segmentation 
foreground = cv2.createBackgroundSubtractorMOG2(detectShadows =False)
topic="kafka_video"
def publish_video(video_file):
    #run producer.py
    producer = KafkaProducer(bootstrap_servers='localhost:9092')
    #define videoCapture
    cap= cv2.VideoCapture(video_file)
    print('publishing video ...')
    while(cap.isopened()):
        success, camframe= cap.read()
        #make sure it works
        if not success:
            print("bad read!")
			break
		# Convert image to jpg 
		ret, buffer= cv2.imencode('.jpg', camframe)
		#break them up into bytes for kafka 
		producer.send(topic, buffer.tobytes())
		time.sleep(0.2)
	cap.release()
	print('end of stream')
def publish_camera():
	#run producer.py
	producer = KafkaProducer(bootstrap_servers='localhost:9092')
	camera= cv2.VideoCapture(0)
	camera.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)
	camera.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)
	camera.set(cv2.CAP_PROP_FPS,6)
	try:
		while(True):
			success, camframe=camera.read()
			grayframe = cv2.cvtColor(camframe, cv2.COLOR_BGR2GRAY)
			blurframe = cv2.GaussianBlur(camframe, (5, 5), 0)
			#Capture motion 
			motionframe = foreground.apply(grayframe)
			detect = (np.sum(motionframe))//255
			if detect > 30:
				print("object in motion size = ", detect)
				# Save stream to .avi file
				out.write(grayframe)
			ret, buffer = cv2.imencode('.jpg',grayframe)
			producer.send(topic, buffer.tobytes())
			# give some time for processing
			time.sleep(0.2)
	except :
		print("\nWe are done.")
		sys.exit(1)
	camera.release()
	out.release()
if __name__ == '__main__' :
	if(len(sys.argv) > 1):
		video_path = sys.argv[1]
		publish_video(video_path)
	else :
		print("we are live ")
publish_camera()

In [ ]:
!pip install kafka-python

     |████████████████████████████████| 256kB 5.0MB/s 


In [ ]:
import sys
import time 
import kafka 
import kafkaProducer 
import numpy as np 
import cv2 
# Define the codec and create VideoWriter object (note isColor is False for Gray) 
fourcc = cv2.VideoWriter_fourcc(*'DIVX') 
out = cv2.VideoWriter('motiondetect.avi',fourcc, 60.0, (1920,1080), isColor = False) 
#MOG2 segmentation  
foreground = cv2.createBackgroundSubtractorMOG2(detectShadows =False) 
topic="kafka_video" 
def publish_video(video_file):
  #run producer.py
  producer = KafkaProducer(bootstrap_servers='localhost:9092')
  #define videoCapture
  cap= cv2.VideoCapture(video_file)
  print('publishing video ...')
  while(cap.isOpened()):
    success, camframe = cap.read()
    #make sure it works
    if not success:
      print("bad read!")
      break
    # Convert image to jpg
    ret, buffer = cv2.imencode('.jpg', camframe)
    #break them up into bytes for kafka
    producer.send(topic, buffer.tobytes())
    time.sleep(0.2)
  cap.release()
  print('end of stream') 

def publish_camera():
  #run producer.py
  producer = KafkaProducer(bootstrap_servers='localhost:9092')
  camera = cv2.VideoCapture(0)
  camera.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)
  camera.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)
  camera.set(cv2.CAP_PROP_FPS,60)
  try:
    while(True):
      success, camframe = camera.read()
      grayframe = cv2.cvtColor(camframe, cv2.COLOR_BGR2GRAY)
      blurframe = cv2.GaussianBlur(camframe, (5, 5), 0)
      #Capture motion
      motionframe = foreground.apply(grayframe)
      detect = (np.sum(motionframe))//255
      if detect > 30:
        print("object in motion size = ", detect)
        # Save stream to .avi file
        out.write(grayframe)
      ret, buffer = cv2.imencode('.jpg',grayframe)
      producer.send(topic, buffer.tobytes())
      # give some time for processing
      time.sleep(0.2)
  except:
    print("\nWe are done.")
    sys.exit(1)
  camera.release()
  out.release() 

if __name__ == '__main__' :
  if (len(sys.argv) > 1):
    video_path = sys.argv[1]
    publish_video(video_path)
  else :
    print("we are live ") 
publish_camera() 

In [ ]:
#code JDID Producer
import sys 
import time  
from kafka import KafkaProducer 
import numpy as np #
import cv2 
# Define the codec and create VideoWriter object (note isColor is False for Gray) 

fourcc = cv2.VideoWriter_fourcc(*'DIVX') 
#creating a video motiondetect that contains only the detected motion in 60 fram per second

out = cv2.VideoWriter('motiondetect.avi',fourcc, 60.0, (1920,1080), isColor = False) 
#MOG2 segmentation  
#this is the algorithme that capture the motion
#this is opencv algorithm choosing 
foreground = cv2.createBackgroundSubtractorMOG2(detectShadows =False) 
#that's how the producer and consumer can talk to each other
#topic create how the producer and consumer can talk to each other
topic="kafka_video" 
def publish_video(video_file): 
  #create producer with this parametres
  producer = KafkaProducer(bootstrap_servers='localhost:9092') 
  #define videoCapture 
  cap= cv2.VideoCapture(video_file) 
  print('publishing video ...') 
  while(cap.isopened()):
    success,camframe=cap.read() 
    #make sure it works 
    if not success:
      print("bad read!")
      break 
    #Convert image to jpg
    ret, buffer= cv2.imencode('.jpg', camframe) 
    #break them up into bytes for kafka
    producer.send(topic, buffer.tobytes())
    time.sleep(0.2) 
    cap.release() 
    print('end of stream') 

def publish_camera(): 
  #run producer.py 
  producer = KafkaProducer(bootstrap_servers='localhost:9092') 
  camera= cv2.VideoCapture(0) 
  camera.set(cv2.CAP_PROP_FRAME_WIDTH, 1920) 
  camera.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080) 
  camera.set(cv2.CAP_PROP_FPS,60)
  try:
    while(True):
      success,camframe=camera.read()
      #camframe come gray
      grayframe = cv2.cvtColor(camframe, cv2.COLOR_BGR2GRAY)
      #work with gray 
      blurframe = cv2.GaussianBlur(camframe, (5, 5), 0)
      #create motion frame by applying foreground
      motionframe = foreground.apply(grayframe)
      detect = (np.sum(motionframe))//255
      if detect > 30:
        print("object in motion size = ", detect)
        # Save stream to .avi file
	# Convert to a jpg image
        out.write(grayframe)
        ret, buffer = cv2.imencode('.jpg',grayframe)
        producer.send(topic, buffer.tobytes())
        # give some time for processing
        time.sleep(0.2)
  except :
    print("\nWe are done.")
    sys.exit(1)
  camera.release()
  out.release() 

if __name__ == '__main__' :
  if(len(sys.argv) > 1):
    video_path = sys.argv[1]
    publish_video(video_path)
  else :
      print("we are live ")
publish_camera()


In [ ]:
#Code Jdid Consumer
import datetime 
from flask import Flask, Response 
from kafka import KafkaConsumer 
#start consumper.py 
#importing the topic for the apache kafka so it can communicate with the #producer 
topic="kafka_video" 
consumer = KafkaConsumer(topic, bootstrap_servers=['localhost:9092']) 
# Get the flask app ready for view 
app = Flask(__name__)
@app.route('/video',methods=['GET']) 
#this function takes the stream function  
def video():
  return Response(get_video_stream(),mimetype='multipart/x-mixed-replace; boundary=frame')

#for each message on the consumer puts all the frames all together again and #creates an image on the video stream  
def get_video_stream():
  for msg in consumer:
    yield (b'--frame\r\n' b'Content-Type: image/jpg\r\n\r\n' + msg.value + b'\r\n\r\n')
if __name__ == "__main__":
  app.run(host='0.0.0.0' , debug=True) 
  #Done

ModuleNotFoundError: ignored

In [ ]:
frame.shape

NameError: ignored